In [1]:
import pandas, pathlib, json

from collections import defaultdict

## Parsing the JSON files produced by EIT Pathogena

We've downloaded the `main_report` and `resistance_prediction_report` for each sample using the CLI tool; now we need to aggregate the relevant data into some pandas DataFrames to make downstream analysis easier.

We will produce two tables.

`EFFECTS`: one row per mutation that leads to a predicted effect on a drug

`PREDICTIONS`: the above collapsed down to one row per sample per drug

In [2]:
predictions = defaultdict(list)
effects = defaultdict(list)

for folder in ['mgit', 'ukmyc']:

    path = pathlib.Path('dat/outputs/')

    for i in (path / folder).glob('*.resistance_prediction_report.json'):

        # Exclude the five samples with read naming issues
        if i.stem.split('.')[0] in ['ERR4796519', 'ERR4796408', 'ERR4796312', 'ERR4796311', 'ERR4796303']:
            print(f"Skipping {i.stem} due to read naming issues")
            continue

        f = open(i)

        sample = i.stem.split('.')[0]

        data = json.load(f)

        for drug, prediction in data['data']['antibiogram'].items():
            predictions['ENA_RUN_ACCESSION'].append(sample)
            predictions['DRUG'].append(drug)
            predictions['PREDICTION'].append(prediction)

        for drug in data['data']['effects']:
            for j in (data['data']['effects'][drug]):
                if "phenotype" in j.keys():
                    continue
                effects['ENA_RUN_ACCESSION'].append(sample)
                effects['DRUG'].append(drug)
                effects['GENE'].append(j['gene'])
                effects['MUTATION'].append(j['mutation'])
                effects['PREDICTION'].append(j['prediction'])
                epistasis = False
                if 'expert_rule' in j['evidence'].keys():
                    if "epistasis" in j['evidence']['expert_rule']:
                        epistasis = True
                effects['EPISTASIS'].append(epistasis)         

predictions= pandas.DataFrame.from_dict(predictions)
effects = pandas.DataFrame.from_dict(effects)

Skipping ERR4796519.resistance_prediction_report due to read naming issues
Skipping ERR4796408.resistance_prediction_report due to read naming issues
Skipping ERR4796311.resistance_prediction_report due to read naming issues
Skipping ERR4796303.resistance_prediction_report due to read naming issues
Skipping ERR4796312.resistance_prediction_report due to read naming issues


In [3]:
effects[:3]

,ENA_RUN_ACCESSION,DRUG,GENE,MUTATION,PREDICTION,EPISTASIS
0,ERR4799790,AMI,rrs,g1484t:14,R,False
1,ERR4799790,CAP,rrs,g1484t:14,R,False
2,ERR4799790,CAP,tlyA,L11L,S,False


In [4]:
effects[effects.EPISTASIS]

,ENA_RUN_ACCESSION,DRUG,GENE,MUTATION,PREDICTION,EPISTASIS
16017,ERR5917746,AMI,None,eis@268_del_acgcatcgccg&eis@c-14t,S,True
16033,ERR5917746,KAN,None,eis@268_del_acgcatcgccg&eis@c-14t,S,True
50139,ERR4829406,AMI,None,eis@626_ins_g&eis@c-14t,S,True
50154,ERR4829406,KAN,None,eis@626_ins_g&eis@c-14t,S,True


In [5]:
def assign_booleans(row):
    minor_call = False
    is_null = False
    if ":" in row.MUTATION:
        minor_call = True
    if 'x' in row.MUTATION:
        is_null = True
    elif 'X' in row.MUTATION:
        is_null = True
    return pandas.Series([minor_call, is_null])

effects[['IS_MINOR_ALLELE','IS_NULL']] = effects.apply(assign_booleans, axis=1) 

pandas.crosstab(effects.IS_MINOR_ALLELE, effects.IS_NULL)

IS_NULL,False,True
IS_MINOR_ALLELE,,
False,51842,503
True,3118,0


In [6]:
print(f"The EFFECTS table contains {effects.ENA_RUN_ACCESSION.nunique()} samples and {effects.shape[0]} rows")

The EFFECTS table contains 2658 samples and 55463 rows


In [7]:
predictions[:3]

,ENA_RUN_ACCESSION,DRUG,PREDICTION
0,ERR4799790,AMI,R
1,ERR4799790,BDQ,S
2,ERR4799790,CAP,R


In [8]:
print(f"The PREDICTIONS table contains {predictions.ENA_RUN_ACCESSION.nunique()} samples and {predictions.shape[0]} rows, the latter being 15 times the former since the WHOv2 catalogue contains 15 drugs")

The PREDICTIONS table contains 2658 samples and 39870 rows, the latter being 15 times the former since the WHOv2 catalogue contains 15 drugs


In [9]:
effects.set_index(['ENA_RUN_ACCESSION','DRUG','GENE','MUTATION'], inplace=True, verify_integrity=True)
predictions.set_index(['ENA_RUN_ACCESSION','DRUG'], inplace=True, verify_integrity=True)


In [10]:
effects[:3]

PREDICTION  EPISTASIS  IS_MINOR_ALLELE  \
ENA_RUN_ACCESSION DRUG GENE MUTATION                                           
ERR4799790        AMI  rrs  g1484t:14          R      False             True   
                  CAP  rrs  g1484t:14          R      False             True   
                       tlyA L11L               S      False            False   

                                       IS_NULL  
ENA_RUN_ACCESSION DRUG GENE MUTATION            
ERR4799790        AMI  rrs  g1484t:14    False  
                  CAP  rrs  g1484t:14    False  
                       tlyA L11L         False

In [11]:
predictions[:3]

PREDICTION
ENA_RUN_ACCESSION DRUG           
ERR4799790        AMI           R
                  BDQ           S
                  CAP           R

In [12]:
effects.to_csv('dat/RAW_EFFECTS.csv')
predictions.to_csv('dat/RAW_PREDICTIONS.csv')